In [1]:
import os

os.chdir("..")

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn


In [2]:
from data.data_generator import get_data, preprocess
from src.my_dnn import create_dnn
from sklearn.model_selection import train_test_split
import numpy as np
import keras
import keras_tuner as kt

In [3]:
def regression_func(x):
    return np.exp(np.linalg.norm(x, axis=1))

input_dim = 7

x, y = get_data(regression_func, x_dim=input_dim, num_samples=100, sigma=0.05)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

train_data = preprocess(x_train, y_train, batch_size=20, training=True)
val_data = preprocess(x_val, y_val, batch_size=20, training=False)

In [4]:
def model_builder(hp):
    hp_units = hp.Int("units", min_value=10, max_value=20, step=10)
    hp_layers = hp.Int("layers", min_value=10, max_value=20, step=10)
    hp_nets = hp.Int("networks", min_value=80, max_value=2 * 80, step=80)
    # hp_beta = hp.Float('beta', min_value=20, max_value=80)
    # hp_gamma = hp.Float('gamma', min_value=30, max_value=80)
    hp_learning_rate = hp.Float(
        "learning_rate", min_value=np.exp(-16), max_value=np.exp(-14), sampling="log"
    )

    # Hier nutzen Sie Ihre angepasste `create_dnn` Funktion mit den hp-Argumenten
    model = create_dnn(
        train_shape=x_train.shape,  # Beispielwerte
        num_networks=hp_nets,
        num_layers=hp_layers,
        num_neurons=hp_units,
        beta=80,
        gamma=80,
        delta=1,
    )

    optimizer = keras.optimizers.SGD(learning_rate=hp_learning_rate)

    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"]
    )

    return model

In [7]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

tuner = kt.RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=2,
    directory='my_dir',
    project_name='keras_tuner_demo'
)

# Starten des Tuning-Prozesses
# Hinweis: Ersetzen Sie `x_train`, `y_train`, `x_val`, `y_val` durch Ihre Daten.
tuner.search(x_train, y_train, epochs=100, validation_data=(x_val, y_val))

# Abrufen der besten Hyperparameter
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Die besten Hyperparameter sind:
- Anzahl der Netzwerke: {best_hps.get('num_networks')}
- Anzahl der Schichten: {best_hps.get('num_layers')}
- Anzahl der Neuronen: {best_hps.get('num_neurons')}
- Beta: {best_hps.get('beta')}
- Gamma: {best_hps.get('gamma')}
- Delta: {best_hps.get('learning_rate')}
""")


Trial 12 Complete [01h 37m 03s]
val_loss: 1.8369200825691223

Best val_loss So Far: 1.3565861582756042
Total elapsed time: 16h 07m 00s

Search: Running Trial #13

Value             |Best Value So Far |Hyperparameter
20                |20                |units
20                |10                |layers
160               |160               |networks
4.756e-07         |4.1569e-07        |learning_rate

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 49s 19s/step - mean_squared_error: 15.7969 - loss: 15.0494 - val_loss: 2.1089 - val_mean_squared_error: 2.1089
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 52s 20s/step - mean_squared_error: 1.6385 - loss: 1.6017 - val_loss: 1.9544 - val_mean_squared_error: 1.9544
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 72s 14s/step - mean_squared_error: 1.3529 - loss: 1.3421 - val_loss: 2.0078 - val_mean_squared_error: 2.0078
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 41s 14s/step - mean_squared_error: 1.3430 - loss: 1.3274 - val_loss: 1.9661 - val_mean_squared_error: 1.9661
Epoch 5/1

In [11]:
x_val, y_val = get_data(m1, x_dim=m1.expected_dim, num_samples=10**5)
validation_data = preprocess(x_val, y_val, training=False)

model.evaluate(validation_data)

1563/1563 [==============================] - 26s 17ms/step - loss: 0.0071 - mean_squared_error: 0.0071


[0.007052791304886341, 0.007052791304886341]

In [15]:


def test_norm_l1(model):
    # Test if L1 projection of last layer worked
    weights = tf.reshape(model.trainable_variables[-1], [-1])
    norm = tf.norm(weights, ord=1)
    print("norm: {}, gamma: {}".format(norm, model.gamma))


def test_norm_l2(model):
    # Test if L2 projection of inner weights worked
    current_weights = tf.concat(
        [tf.reshape(v, [-1]) for v in model.trainable_weights[:-1]], axis=0
    )
    sub_nets_init_weights = model.sub_nets_init_weights
    diff = sub_nets_init_weights - current_weights
    norm = tf.norm(diff)
    print("norm: {}, delta: {}".format(norm, model.delta))

In [16]:
test_norm_l1(model)
test_norm_l2(model)

norm: 0.7622666954994202, gamma: 10
norm: 0.0030101335141807795, delta: 1.0
